# 0. Dataset

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import MDAnalysis as mda
import pandas as pd
from MDAnalysis.analysis import distances
from scipy.linalg import expm
import os
from IPython.display import clear_output

## Data Preprocessing

In [21]:
print('reading files...')

# Read the tab separated file from BioLip
df = pd.read_csv('BioLiP.csv')

# list all PDB structure files
os.system('ls data/PDB_files > all_pdb_files.txt')

print('removing entry with unavailable PDB structures...')

# filter for only the data PDB structure available
avail_list = []
f = open('all_pdb_files.txt','r')
for line in f:
    avail_list.append(line[3:7])
    
avail = []
for pdb_id in df.PDB_ID:
    if pdb_id in avail_list:
        avail.append(1)
    else:
        avail.append(0)
        
# drop data with unavailable structures
df['available'] = avail
df = df[df.available==1]
df = df.reset_index(drop=True)

print('get protein sequences and length from FASTA...')

# get peptide sequences
peptide_seq = []
len_peptide_seq = []

# Read the FASTA sequence files of all filtered PDB entries
# get the peptide sequence and length to the list for each entry
# if peptide not found just add X
for i in range( len(df) ):
    found = 0
    pdb_id = df.iloc[i,:].PDB_ID    
    # seeking for peptide chain
    f = open('data/FASTA_files/%s.fasta'%pdb_id)
    for line in f:
        if( line.split()[0][0] == '>' and
            line.split()[0][-1] == df.iloc[i,:].peptide_chain_ID):
            peptide_seq.append(f.readline()[:-1])
            found = 1
            break
    if found == 0:
        peptide_seq.append('X')
    f.close()
    len_peptide_seq.append(len(peptide_seq[-1]))
            
# add the FASTA sequence column
df['peptide_seq'] = peptide_seq
df['len_peptide_seq'] = len_peptide_seq

# Then determine the length of receptors and add a column into df
len_protein_seq = []
for i in range( len(df) ):
    len_protein_seq.append(len(df.iloc[i,:]["Receptor sequence"]))    
df['len_protein_seq'] = len_protein_seq

# ****filter for only data with 3-30 amino acid peptide****
df = df[df.len_peptide_seq <= 30]
df = df[df.len_peptide_seq >= 3].reset_index(drop=True)

# ****filter for only data with 30-300 amino acid receptor protein****
df = df[df.len_protein_seq <= 300]
df = df[df.len_protein_seq >= 30].reset_index(drop=True)

print('dropping data with unknown sequences...')

# drop PDB with incomplete structural data
incomplete_entry = []
for i in range( len(df) ):
    if 'X' in df.iloc[i,:].bind_res_renum:
        incomplete_entry.append(i)
df = df.drop(incomplete_entry).reset_index(drop=True)

# other incomplete entry with missing residues
incomplete = ['3gi0']
n_entry = len(df)

for entry in range( n_entry ):
    pdb_id = df.PDB_ID[entry]
    os.system('cp data/PDB_files/pdb'+pdb_id+'.ent protein_temp.pdb')

    Receptor_chain = df.Receptor_chain[entry]
    len_protein_seq = df.len_protein_seq[entry]

    # import a pdb file as the universe
    PDB = 'protein_temp.pdb'
    u = mda.Universe(PDB,PDB)

    # select only groups of receptor chain if receptor is longer than 50 aa
    # otherwise select the whole protein
    #if len(df['Receptor sequence'][entry]) > 50:
        #CA = u.select_atoms('protein and name CA and chainID '+str(Receptor_chain))
    #else:
        #CA = u.select_atoms('protein and name CA and not chainID '+str(peptide_chain_ID))
    CA = u.select_atoms('protein and name CA and chainID '+str(Receptor_chain))
    CA.write("CA.pdb")

    # create the amino acid sequence list
    seq = CA.atoms.resnames
    
    if len_protein_seq != len(seq):
        incomplete.append(pdb_id)

    # list of atom index with alternate locations (altLoc)
    for i in range(len(CA)):
        if CA[i].altLoc != '' and CA[i].altLoc != 'A':
            incomplete.append(pdb_id)
            break
    clear_output(wait=True)
    print('checked incomplete PDB structures: entry '+str(entry)+'/'+str(n_entry))
    
incomplete_entry = []
for pdb_id in incomplete:
    incomplete_entry += list(df[df.PDB_ID==pdb_id].index)

df = df.drop(incomplete_entry).reset_index(drop=True)

print('finalizing primary dataset...')

# drop unused columns 
# then assign protein-peptide interaction entry NO.  
df = df[["PDB_ID","Receptor_chain","peptide_chain_ID","bind_res_renum","Receptor sequence","peptide_seq","len_protein_seq","len_peptide_seq"]]
df = df.reset_index().rename(columns={"index":"entry"})


checked incomplete PDB structures: entry 18531/18532
finalizing primary dataset...


In [22]:
df

,entry,PDB_ID,Receptor_chain,peptide_chain_ID,bind_res_renum,Receptor sequence,peptide_seq,len_protein_seq,len_peptide_seq
0,0,1a0n,B,A,Y8 Y10 W36 P51 N53 Y54,VTLFVALYDYEARTEDDLSFHKGEKFQILNSSEGDWWEARSLTTGE...,PPRPLPVAPGSSKT,58,14
1,1,1a1m,A,C,Y7 Y9 N63 Y74 N77 Y84 R97 Y99 D114 Y123 T143 K...,GSHSMRYFYTAMSRPGRGEPRFIAVGYVDDTQFVRFDSDAASPRTE...,TPYDINQML,278,9
2,2,1a1n,A,C,R62 N63 I66 F67 T69 T73 Y74 S77 N80 Y84 R97 Y9...,GSHSMRYFYTAMSRPGRGEPRFIAVGYVDDTQFVRFDSDAASPRTE...,VPLRPMTY,276,8
3,3,1a1o,A,C,Y7 R62 N63 I66 T69 N77 I80 Y84 R97 Y99 Y123 T1...,GSHSMRYFYTAMSRPGRGEPRFIAVGYVDDTQFVRFDSDAASPRTE...,KPIVQYDNF,276,9
4,4,1a1r,A,C,V1 G3 E4 V5 Q6 I7 V8 S9 T35 I36 A37 P60 G62,VEGEVQIVSTATQTFLATCINGVCWTVYHGAGTRTIASPKGPVIQM...,GSVVIVGRIVLSGKPA,152,16
...,...,...,...,...,...,...,...,...,...
11901,11901,6vrn,E,P,R29 Y30,AGITQSPRHKVTETGTPVTLRCHQTENHRYMYWYRQDPGHGLRLIH...,HMTEVVRHC,242,9
11902,11902,6vxy,A,C,F24 H40 N79 Y131 Q155 S172 C173 Q174 G175 S177...,IVGGYTCGANTVPYQVSLNSGYHFCGGSLINSQWVVSAAHCYKSGI...,GRGTKSIPPIAFPD,223,14
11903,11903,6w05,H,P,F31 G32 W51 H52 F58 D98 S99 Y102 D103 H104 S107,VQLVESGGGVVQPGRSLRLSCAASGFTFRNFGMHWVRQTPGKGLEW...,NPNANPNA,224,8
11904,11904,6w05,L,P,Y91 N92 N93 G94,QIVMTQSPATVSVSPGERATLSCRASRSVTSKLAWYQQKPGQAPRL...,NPNANPNA,212,8
